In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/skin-disease-book-2/Andrews-Diseases-of-the-Skin-Clinical-Dermatology.pdf


In [2]:
pip install faiss-cpu sentence-transformers pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 63.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 85.6 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
import faiss
import numpy as np
import fitz  # PyMuPDF
from sentence_transformers import SentenceTransformer
import re

In [5]:
model = SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"  # Extract text from each page
    # Split into meaningful chunks (sentences/paragraphs)
    sentences = re.split(r'\.\s+', text)  # Split by period (.)
    return [s.strip() for s in sentences if len(s.strip()) > 10]  # Remove short sentences


In [8]:
def save_vector_database(pdf_path, index_name="book_vector_database"):
    sentences = extract_text_from_pdf(pdf_path)
    embeddings = model.encode(sentences, convert_to_numpy=True)

    # Create FAISS index
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)  # L2 similarity
    index.add(embeddings)

    # Save FAISS index
    faiss.write_index(index, index_name)

    print(f"Vector database saved as '{index_name}' with {len(sentences)} entries.")


In [13]:

import os

pdf_dir = "/kaggle/input/skin-disease-book-2"
print(os.listdir(pdf_dir))  # This will show the actual PDF filename


['Andrews-Diseases-of-the-Skin-Clinical-Dermatology.pdf']


In [16]:
pdf_file = "/kaggle/input/skin-disease-book-2/Andrews-Diseases-of-the-Skin-Clinical-Dermatology.pdf"
save_vector_database(pdf_file)

Batches:   0%|          | 0/1676 [00:00<?, ?it/s]

Vector database saved as 'book_vector_database' with 53624 entries.


In [3]:
import fitz  # PyMuPDF
from sentence_transformers import SentenceTransformer
import faiss
import json
from langchain.text_splitter import RecursiveCharacterTextSplitter

def extract_and_chunk_text(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    
    # Better chunking
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,
        chunk_overlap=50,
        separators=["\n\n", "\n", ". ", " ", ""]
    )
    return text_splitter.split_text(text)

def save_vector_database(pdf_path, index_name="book_vector_db"):
    chunks = extract_and_chunk_text(pdf_path)
    model = SentenceTransformer("BAAI/bge-base-en")  # Domain-aware model
    embeddings = model.encode(chunks, convert_to_numpy=True)

    # Create optimized FAISS index
    dimension = embeddings.shape[1]
    index = faiss.IndexHNSWFlat(dimension, 32)
    index.add(embeddings)
    faiss.write_index(index, f"{index_name}.faiss")

    # Save metadata
    metadata = [{"id": i, "text": chunk, "source": pdf_path} for i, chunk in enumerate(chunks)]
    with open(f"{index_name}_metadata.json", "w") as f:
        json.dump(metadata, f)

    print(f"Saved FAISS index ({len(chunks)} chunks) and metadata.")

# Example usage
pdf_file = "/kaggle/input/skin-disease-book-2/Andrews-Diseases-of-the-Skin-Clinical-Dermatology.pdf"
save_vector_database(pdf_file)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/719 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/655 [00:00<?, ?it/s]

Saved FAISS index (20940 chunks) and metadata.


In [1]:
pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.9/433.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: async-timeout
    Found existing installation: async-timeout 5.0.1
    Uninstalling async-timeout-5.0.1:
      Successfully uninstalled async-timeout-5.0.1
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.3
    Uninstalling langchain-text-splitters-0.3.3:
      Successfully uninstalled langchain-text-splitters-0.3.3
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
  

In [2]:
pip install pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 84.3 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 20.5 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
import os
import pickle
import json
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings

# Step 1: Load PDF
pdf_path =  "/kaggle/input/skin-disease-book-2/Andrews-Diseases-of-the-Skin-Clinical-Dermatology.pdf" # Change this to your PDF file
loader = PyMuPDFLoader(pdf_path)
pages = loader.load()

# Step 2: Split text
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150
)
docs = text_splitter.split_documents(pages)

# Step 3: Embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Step 4: Create FAISS vector store
vectorstore = FAISS.from_documents(docs, embedding_model)

# Save vectorstore (FAISS index)
vectorstore.save_local("book_vector_database")

# Step 5: Save with .pkl extension
with open("book_vector_database.pkl", "wb") as f:
    pickle.dump(vectorstore, f)

# Step 6: Save metadata
metadata = [doc.metadata for doc in docs]
with open("book_vector_database_metadata.json", "w", encoding="utf-8") as f:
    json.dump(metadata, f, indent=4)
